# Aircraft Crashes Data Collection And Cleaning

## Overview

This notebook collects and prepares the data for the analysis of all the aircraft accidents since 1918.

### About dataset

The data will be scraped from the [BAAA Crash Archives](https://www.baaa-acro.com/crash-archives) and the [ASN Database](https://asn.flightsafety.org/database/).

**BAAA**

`date` date and local time of the accident<br>
`aircraft_type` aircraft make and model<br>
`operator` operator of the aircraft<br>
`registration` unique code to a single aircraft, required by international convention<br>
`flight_phase` phase of the flight when the accident occured<br>
`flight_type` type of flight (ex: military)<br>
`survivors` indicates if there was survivors or not<br>
`site` type of location where the accident happened (ex: mountains)<br>
`departure` city where the departure was planned<br> 
`arrival` city where the arrival was planned<br> 
`msn` manufacturer's serial number of the aircraft<br>
`yom` year of manufacture of the aircraft involved in the accident<br>
`flight_number` flight number<br>
`location` location of the accident<br>
`country` country where the crash happened<br>
`region` region of the world where the crash happened<br>
`crew_on_board` number of crew members on board at the time of the accident<br>
`crew_fatalities` number of crew members who died in the crash<br>
`pax_on_board` number of passengers on board at the time of the accident<br> 
`pax_fatalities` number of passengers who died in the crash<br>                 
`other_fatalities` other victims of the accident outside of the aircraft<br>
`total_fatalities` total number of deaths<br>
`captain_flying_hours` number of flying hours of the captain<br>
`captain_flying_hours_on_type` number of hours the captain flew on the type of aircraft involved in the crash<br>
`copilot_flying_hours` number of flying hours of the copilot<br>  
`copilot_flying_hours_on_type` number of hours the copilot flew on the type of aircraft involved in the crash<br>  
`aircraft_flying_hours` number of flying hours of the aircraft before the crash<br>
`aircraft_flight_cycles` number of flights of the aircraft<br><br>


**ASN**

`date` date of the accident<br>
`time` time of the accident<br>
`type` make and model of the aircraft<br>
`first_flight` year the aircraft was inaugurated<br>
`engine` type and number of engines<br>
`owner` operator of the aircraft<br>
`registration` unique code to a single aircraft, required by international convention<br>
`msn` manufacturer's serial number of the aircraft<br>
`year_of_manufacture` year of manufacture of the aircraft involved in the accident<br>
`total_airframe_hrs` number of flying hours of the aircraft before the crash<br>
`cycles` number of flights of the aircraft<br>
`engine_model` make and model of the aircraft engine<br>
`fatalities` total number of fatalities<br>
`occupants` number of crew members and passengers on board<br>
`other_fatalities` other victims of the accident outside of the aircraft<br>
`aircraft_damage` severity of the aircraft damage
`category` type of accident<br>
`location` location of the crash<br>
`phase` phase of the flight when the accident occured<br>
`nature` type of flight (ex: military)<br>
`departure_airport` airport where the departure was planned<br>
`destination_airport` airport when the arrival was planned<br>
`investigating_agency` agency who made the accident deport<br>
`confidence_rating` quality of the information (ex: missing information)

---

## Data Collection

In [1]:
from bs4 import BeautifulSoup
from datetime import datetime
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import math
import numpy as np
import pandas as pd
import pickle
import re
import requests
from urllib.parse import unquote

### BAAA

### ASN

---

## Data Exploration

### BAAA

In [2]:
baaa_df = pd.read_csv('data/baaa_scraped_data.csv')
baaa_df.head()

,date,aircraft_type,operator,registration,flight_phase,flight_type,survivors,site,schedule,msn,...,pax_on_board,pax_fatalities,other_fatalities,total_fatalities,captain_flying_hours,captain_flying_hours_on_type,copilot_flying_hours,copilot_flying_hours_on_type,aircraft_flying_hours,aircraft_flight_cycles
0,"Mar 17, 2025 at 1818 LT",BAe Jetstream 31,Línea Aérea Nacional de Honduras - LANHSA,HR-AYW,Takeoff (climb),Scheduled Revenue Flight,Yes,"Lake, Sea, Ocean, River",Roatán – La Ceiba,863,...,15.0,10.0,0.0,13,NaN,NaN,NaN,NaN,NaN,NaN
1,"Mar 13, 2025 at 0733 LT",Cessna 525 CitationJet CJ2,LBL 525 CZ LLC,N525CZ,Takeoff (climb),Private,No,"Plain, Valley",Mesquite - Addison,525A-0380,...,0.0,0.0,0.0,1,NaN,NaN,NaN,NaN,NaN,NaN
2,"Mar 7, 2025",Antonov AN-32,Indian Air Force - Bharatiya Vayu Sena,NaN,Landing (descent or approach),Military,Yes,Airport (less than 10 km from airport),NaN,NaN,...,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,"Mar 4, 2025 at 0954 LT",BAe Jetstream 31,SAETA Perú (Servicios Aéreos Tarapota),OB-2178,Landing (descent or approach),Scheduled Revenue Flight,Yes,Airport (less than 10 km from airport),Iquitos - Güeppí,861,...,11.0,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,"Feb 25, 2025",Antonov AN-26,Sudanese Air Force - Al Quwwat al-Jawwiya As-S...,NaN,Takeoff (climb),Military,No,City,NaN,NaN,...,13.0,13.0,29.0,46,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
baaa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36087 entries, 0 to 36086
Data columns (total 27 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   date                          36087 non-null  object 
 1   aircraft_type                 36087 non-null  object 
 2   operator                      36085 non-null  object 
 3   registration                  34900 non-null  object 
 4   flight_phase                  35476 non-null  object 
 5   flight_type                   36030 non-null  object 
 6   survivors                     34811 non-null  object 
 7   site                          35720 non-null  object 
 8   schedule                      25713 non-null  object 
 9   msn                           28065 non-null  object 
 10  yom                           26337 non-null  float64
 11  flight_number                 2896 non-null   object 
 12  location                      36076 non-null  object 
 13  c

In [4]:
baaa_df.isnull().sum()

date                                0
aircraft_type                       0
operator                            2
registration                     1187
flight_phase                      611
flight_type                        57
survivors                        1276
site                              367
schedule                        10374
msn                              8022
yom                              9750
flight_number                   33191
location                           11
country                             3
region                              2
crew_on_board                      22
crew_fatalities                     1
pax_on_board                       50
pax_fatalities                      4
other_fatalities                   16
total_fatalities                    0
captain_flying_hours            29207
captain_flying_hours_on_type    30242
copilot_flying_hours            33856
copilot_flying_hours_on_type    34097
aircraft_flying_hours           30384
aircraft_fli

In [5]:
# Check for duplicates
baaa_df[baaa_df.duplicated(keep=False)]

,date,aircraft_type,operator,registration,flight_phase,flight_type,survivors,site,schedule,msn,...,pax_on_board,pax_fatalities,other_fatalities,total_fatalities,captain_flying_hours,captain_flying_hours_on_type,copilot_flying_hours,copilot_flying_hours_on_type,aircraft_flying_hours,aircraft_flight_cycles
2500,"Jun 15, 2008",Harbin Yunsunji Y-12,China Flying Dragon Special Aviation Company,B-3841,Flight,Geographical / Geophysical / Scientific,Yes,Mountains,NaN,0061,...,2.0,2.0,0.0,3,NaN,NaN,NaN,NaN,NaN,NaN
2501,"Jun 15, 2008",Harbin Yunsunji Y-12,China Flying Dragon Special Aviation Company,B-3841,Flight,Geographical / Geophysical / Scientific,Yes,Mountains,NaN,0061,...,2.0,2.0,0.0,3,NaN,NaN,NaN,NaN,NaN,NaN
7540,"Jun 8, 1988",Lockheed C-130 Hercules,United States Air Force - USAF (since 1947),61-2373,Landing (descent or approach),Training,No,Airport (less than 10 km from airport),Little Rock - Greenville,3720,...,0.0,0.0,0.0,6,NaN,NaN,NaN,NaN,NaN,NaN
7541,"Jun 8, 1988",Lockheed C-130 Hercules,United States Air Force - USAF (since 1947),61-2373,Landing (descent or approach),Training,No,Airport (less than 10 km from airport),Little Rock - Greenville,3720,...,0.0,0.0,0.0,6,NaN,NaN,NaN,NaN,NaN,NaN
7660,"Dec 28, 1987",PZL-Mielec AN-2,Aeroflot - Russian International Airlines,CCCP-02531,Takeoff (climb),Scheduled Revenue Flight,Yes,"Plain, Valley",NaN,1G121-15,...,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33821,"Sep 30, 1933",Avro 594 Avian,Holden's Air Transport Services,VH-UIV,Landing (descent or approach),Cargo,Yes,Airport (less than 10 km from airport),Salamaua – Bulolo,193,...,1.0,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN
35000,"Oct 18, 1928",Douglas M-3,National Air Transport - USA,NC1064,Flight,Postal (mail),No,Mountains,Cleveland – New York,658,...,0.0,0.0,0.0,1,NaN,NaN,NaN,NaN,NaN,NaN
35001,"Oct 18, 1928",Douglas M-3,National Air Transport - USA,NC1064,Flight,Postal (mail),No,Mountains,Cleveland – New York,658,...,0.0,0.0,0.0,1,NaN,NaN,NaN,NaN,NaN,NaN
35540,"Dec 31, 1923",Loening 23 Air Yacht,New York-Newport Air Service,NaN,NaN,Scheduled Revenue Flight,Yes,"Lake, Sea, Ocean, River",NaN,NaN,...,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN


### ASN

In [6]:
asn_df = pd.read_csv('data/asn_scraped_data.csv')
asn_df.head()

,date,time,type,type_details,owner,registration,msn,year_of_manufacture,total_airframe_hrs,cycles,...,other_fatalities,aircraft_damage,category,location,phase,nature,departure_airport,destination_airport,investigating_agency,confidence_rating
0,Saturday 2 August 1919,NaN,Caproni Ca.48,"Caproni Ca.48, First flight: 1919, 3 Piston en...",Caproni,,,1919.0,NaN,NaN,...,0,"Destroyed, written off",Accident,Verona - Italy,En route,Passenger,Venice-Marco Polo Airport (VCE/LIPZ),Milano-Taliedo Airport,NaN,"Information is only available from news, socia..."
1,Monday 11 August 1919,NaN,Felixstowe Fury,"Felixstowe Fury, First flight: 1918, 5 Piston ...",Royal Air Force - RAF,N123,,1918.0,NaN,NaN,...,0,"Destroyed, written off",Accident,off Felixtowe RNAS - United Kingdom,Initial climb,Military,Felixstowe RNAS,Felixstowe RNAS,NaN,NaN
2,Monday 23 February 1920,NaN,Handley Page O/7,"Handley Page Type O, First flight: 1915, 2 Pis...",Handley Page Transport,G-EANV,HP-7,1919.0,NaN,NaN,...,0,"Destroyed, written off",Accident,"Acadia Siding, Cape Province - South Africa",En route,Passenger - Scheduled,NaN,NaN,NaN,NaN
3,Wednesday 25 February 1920,NaN,Handley Page O/400,"Handley Page Type O, First flight: 1915, 2 Pis...",Handley Page Transport,G-EAMC,HP-27,NaN,NaN,NaN,...,0,"Destroyed, written off",Accident,10 km N of El Shereik - Sudan,Unknown,Unknown,Aswan Airport (ASW/HESN),Khartoum-Civil Airport (KRT/HSSS),NaN,NaN
4,Wednesday 30 June 1920,NaN,Handley Page O/400,"Handley Page Type O, First flight: 1915, 2 Pis...",Handley Page Transport,G-EAKE,HP-22,1919.0,NaN,NaN,...,0,"Destroyed, written off",Accident,Östanå - Sweden,En route,Demo/Airshow/Display,Stockholm (unknown airport),Kjeller Air Base (ENKJ),NaN,NaN


In [7]:
asn_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26566 entries, 0 to 26565
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  26566 non-null  object 
 1   time                  12085 non-null  object 
 2   type                  26566 non-null  object 
 3   type_details          26236 non-null  object 
 4   owner                 26501 non-null  object 
 5   registration          26566 non-null  object 
 6   msn                   26566 non-null  object 
 7   year_of_manufacture   20651 non-null  float64
 8   total_airframe_hrs    5635 non-null   object 
 9   cycles                1641 non-null   object 
 10  engine_model          13704 non-null  object 
 11  fatalities            26566 non-null  object 
 12  other_fatalities      26566 non-null  int64  
 13  aircraft_damage       26566 non-null  object 
 14  category              25765 non-null  object 
 15  location           

In [8]:
asn_df.isnull().sum()

date                        0
time                    14481
type                        0
type_details              330
owner                      65
registration                0
msn                         0
year_of_manufacture      5915
total_airframe_hrs      20931
cycles                  24925
engine_model            12862
fatalities                  0
other_fatalities            0
aircraft_damage             0
category                  801
location                    0
phase                       0
nature                      9
departure_airport        8265
destination_airport      8318
investigating_agency    19919
confidence_rating       14864
dtype: int64

In [9]:
# Check for duplicates
asn_df[asn_df.duplicated(keep=False)]

,date,time,type,type_details,owner,registration,msn,year_of_manufacture,total_airframe_hrs,cycles,...,other_fatalities,aircraft_damage,category,location,phase,nature,departure_airport,destination_airport,investigating_agency,confidence_rating
499,Friday 10 May 1940,NaN,Junkers Ju-52/3m,"Junkers Ju-52/3m, First flight: 1932, 3 Piston...",Luftwaffe,,,NaN,NaN,NaN,...,0,"Destroyed, written off",Accident,Waalhaven - Netherlands,Unknown,Military,NaN,NaN,NaN,Little or no information is available
500,Friday 10 May 1940,NaN,Junkers Ju-52/3m,"Junkers Ju-52/3m, First flight: 1932, 3 Piston...",Luftwaffe,,,NaN,NaN,NaN,...,0,"Destroyed, written off",Accident,Waalhaven - Netherlands,Unknown,Military,NaN,NaN,NaN,Little or no information is available
501,Friday 10 May 1940,NaN,Junkers Ju-52/3m,"Junkers Ju-52/3m, First flight: 1932, 3 Piston...",Luftwaffe,,,NaN,NaN,NaN,...,0,"Destroyed, written off",Accident,Waalhaven - Netherlands,Unknown,Military,NaN,NaN,NaN,Little or no information is available
502,Friday 10 May 1940,NaN,Junkers Ju-52/3m,"Junkers Ju-52/3m, First flight: 1932, 3 Piston...",Luftwaffe,,,NaN,NaN,NaN,...,0,"Destroyed, written off",Accident,Waalhaven - Netherlands,Unknown,Military,NaN,NaN,NaN,Little or no information is available
503,Friday 10 May 1940,NaN,Junkers Ju-52/3m,"Junkers Ju-52/3m, First flight: 1932, 3 Piston...",Luftwaffe,,,NaN,NaN,NaN,...,0,"Destroyed, written off",Accident,Waalhaven - Netherlands,Unknown,Military,NaN,NaN,NaN,Little or no information is available
504,Friday 10 May 1940,NaN,Junkers Ju-52/3m,"Junkers Ju-52/3m, First flight: 1932, 3 Piston...",Luftwaffe,,,NaN,NaN,NaN,...,0,"Destroyed, written off",Accident,Waalhaven - Netherlands,Unknown,Military,NaN,NaN,NaN,Little or no information is available
505,Friday 10 May 1940,NaN,Junkers Ju-52/3m,"Junkers Ju-52/3m, First flight: 1932, 3 Piston...",Luftwaffe,,,NaN,NaN,NaN,...,0,"Destroyed, written off",Accident,Rijksweg 13 - Netherlands,Unknown,Military,NaN,NaN,NaN,Little or no information is available
509,Friday 10 May 1940,NaN,Junkers Ju-52/3m,"Junkers Ju-52/3m, First flight: 1932, 3 Piston...",Luftwaffe,,,NaN,NaN,NaN,...,0,"Destroyed, written off",Accident,Rotterdam - Netherlands,Unknown,Military,NaN,NaN,NaN,Little or no information is available
514,Friday 10 May 1940,NaN,Junkers Ju-52/3m,"Junkers Ju-52/3m, First flight: 1932, 3 Piston...",Luftwaffe,,,NaN,NaN,NaN,...,0,"Destroyed, written off",Accident,"Delft - Den Haag, on the road - Netherlands",Unknown,Military,NaN,NaN,NaN,Little or no information is available
515,Friday 10 May 1940,NaN,Junkers Ju-52/3m,"Junkers Ju-52/3m, First flight: 1932, 3 Piston...",Luftwaffe,,,NaN,NaN,NaN,...,0,"Destroyed, written off",Accident,"Delft - Den Haag, on the road - Netherlands",Unknown,Military,NaN,NaN,NaN,Little or no information is available


---

## Data Cleaning

In [10]:
# Remove duplicates
baaa_df = baaa_df.drop_duplicates()
asn_df = asn_df.drop_duplicates()

In [11]:
# Strip whitespaces
def remove_whitespaces(df):
	for column in df.columns:
		if df[column].dtype == 'object':
			df[column] = df[column].str.strip()
	return df

baaa_df = remove_whitespaces(baaa_df)
asn_df = remove_whitespaces(asn_df)

### Merge dataframes on date and registration number

Although it's not very likely, the same aircraft can be involved in multiple accidents. Combining the registration number and the date ensures the unicity of the rows.

The main (left) dataset will be the one from BAAA as it's the most reliable and the second (right) one will be ASN dataset.

In [12]:
# Convert BAAA date to datetime
baaa_df['date'] = pd.to_datetime(baaa_df['date'], format='%b %d, %Y at %H%M LT', errors='coerce') \
				.fillna(pd.to_datetime(baaa_df['date'], format='%b %d, %Y', errors='coerce'))
assert baaa_df['date'].isna().sum() == 0

In [13]:
# Convert ASN date and time to datetime
asn_df['date'] = pd.to_datetime(asn_df['date'], format='%A %d %B %Y', errors='coerce')

In [ ]:
# Keep data after 1945 (after World War II)
baaa_df = baaa_df[baaa_df['date'].dt.year > 1945]
asn_df = asn_df[asn_df['date'].dt.year > 1945]

In [15]:
# Create date string column
baaa_df['date_str'] = baaa_df['date'].dt.strftime('%Y-%m-%d')
asn_df['date_str'] = asn_df['date'].dt.strftime('%Y-%m-%d')

In [16]:
# Merge two dataframes
df = pd.merge(left=baaa_df, right=asn_df, how='left', on=['registration', 'date_str'])
df.head()

,date_x,aircraft_type,operator,registration,flight_phase,flight_type,survivors,site,schedule,msn_x,...,other_fatalities_y,aircraft_damage,category,location_y,phase,nature,departure_airport,destination_airport,investigating_agency,confidence_rating
0,2025-03-17 18:18:00,BAe Jetstream 31,Línea Aérea Nacional de Honduras - LANHSA,HR-AYW,Takeoff (climb),Scheduled Revenue Flight,Yes,"Lake, Sea, Ocean, River",Roatán – La Ceiba,863,...,0.0,Destroyed,Accident,off Juan Manuel Gálvez International Airport (...,Initial climb,Passenger,Roatán-Juan Manuel Gálvez International Airpor...,La Ceiba-Goloson International Airport (LCE/MHLC),NaN,"Information is only available from news, socia..."
1,2025-03-13 07:33:00,Cessna 525 CitationJet CJ2,LBL 525 CZ LLC,N525CZ,Takeoff (climb),Private,No,"Plain, Valley",Mesquite - Addison,525A-0380,...,0.0,Destroyed,Accident,"near Mesquite Metro Airport (KHQZ), Mesquite, ...",Initial climb,Ferry/positioning,"Mesquite Metro Airport, TX (KHQZ)","Dallas-Addison Airport, TX (ADS/KADS)",NTSB,"Information is only available from news, socia..."
2,2025-03-07 00:00:00,Antonov AN-32,Indian Air Force - Bharatiya Vayu Sena,NaN,Landing (descent or approach),Military,Yes,Airport (less than 10 km from airport),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-03-04 09:54:00,BAe Jetstream 31,SAETA Perú (Servicios Aéreos Tarapota),OB-2178,Landing (descent or approach),Scheduled Revenue Flight,Yes,Airport (less than 10 km from airport),Iquitos - Güeppí,861,...,0.0,Destroyed,Accident,Güeppi Airport (SPGP) - Peru,Landing,Passenger,Iquitos-Coronel FAP Francisco Secada Vignetta ...,Güeppi Airport (SPGP),NaN,"Information is only available from news, socia..."
4,2025-02-25 00:00:00,Antonov AN-26,Sudanese Air Force - Al Quwwat al-Jawwiya As-S...,NaN,Takeoff (climb),Military,No,City,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Add latitude and longitude

In [ ]:
# Merge location (BAAA, then ASN, then country)
df['location'] = df['location_x'].fillna(df['location_y']).fillna(df['country'])
df = df.drop(['location_x', 'location_y'], axis=1)
assert df['location'].isnull().sum() == 0

In [8]:
# Get coordinates from geocoder
geolocator = Nominatim(user_agent='aircraft_crashes_analysis')
geocoder = RateLimiter(geolocator.geocode, min_delay_seconds=1)

def get_coord(row, country=True) -> str:
	result = np.nan
	
	try:
		if (country):
			location = geocoder(row['country'], language='en', exactly_one=True)
		else:
			location = geocoder(row['location'], language='en', exactly_one=True)
		
		if (location):
			print('Coordinates: ({}, {})'.format(location.latitude, location.longitude))
			result = str(location.latitude) + ', ' + str(location.longitude)
	except:
		print('An error occured')
	
	return result


In [ ]:
# Inpute missing coordinates with country coordinates
mask = df['lat_lng'].isna()
df.loc[mask, 'lat_lng'] = df[mask].apply(get_coord, country=True, axis=1)

In [10]:
df['lat_lng'].isna().sum()

18

In [11]:
# Export data
df.to_csv('data/merged_data_with_coordinates.csv', index=False)

In [13]:
# Load data
df = pd.read_csv('data/merged_data_with_coordinates.csv', parse_dates=['date_x', 'date_y'])

In [14]:
df.head()

,date_x,aircraft_type,operator,registration,flight_phase,flight_type,survivors,site,schedule,msn_x,...,aircraft_damage,category,phase,nature,departure_airport,destination_airport,investigating_agency,confidence_rating,location,lat_lng
0,2025-03-17 18:18:00,BAe Jetstream 31,Línea Aérea Nacional de Honduras - LANHSA,HR-AYW,Takeoff (climb),Scheduled Revenue Flight,Yes,"Lake, Sea, Ocean, River",Roatán – La Ceiba,863,...,Destroyed,Accident,Initial climb,Passenger,Roatán-Juan Manuel Gálvez International Airpor...,La Ceiba-Goloson International Airport (LCE/MHLC),NaN,"Information is only available from news, socia...",Roatán Islas de la Bahía,"16.34902105, -86.49775125625627"
1,2025-03-13 07:33:00,Cessna 525 CitationJet CJ2,LBL 525 CZ LLC,N525CZ,Takeoff (climb),Private,No,"Plain, Valley",Mesquite - Addison,525A-0380,...,Destroyed,Accident,Initial climb,Ferry/positioning,"Mesquite Metro Airport, TX (KHQZ)","Dallas-Addison Airport, TX (ADS/KADS)",NTSB,"Information is only available from news, socia...","Mesquite Metro, Texas","32.749898900000005, -96.53114976775751"
2,2025-03-07 00:00:00,Antonov AN-32,Indian Air Force - Bharatiya Vayu Sena,NaN,Landing (descent or approach),Military,Yes,Airport (less than 10 km from airport),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Bagdogra, West Bengal","26.6981094, 88.3245465"
3,2025-03-04 09:54:00,BAe Jetstream 31,SAETA Perú (Servicios Aéreos Tarapota),OB-2178,Landing (descent or approach),Scheduled Revenue Flight,Yes,Airport (less than 10 km from airport),Iquitos - Güeppí,861,...,Destroyed,Accident,Landing,Passenger,Iquitos-Coronel FAP Francisco Secada Vignetta ...,Güeppi Airport (SPGP),NaN,"Information is only available from news, socia...","Güeppí, Loreto","-0.1176738, -75.2510798"
4,2025-02-25 00:00:00,Antonov AN-26,Sudanese Air Force - Al Quwwat al-Jawwiya As-S...,NaN,Takeoff (climb),Military,No,City,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Wadi Seidna AFB, Khartoum (الخرطوم)","14.5844444, 29.4917691"


### Split some columns into multiple

In [ ]:
# Split schedule into 2 columns
schedule = df['schedule'].str.split(' - ', expand=True)
df['departure'] = schedule[0]
df['arrival'] = schedule[1]
df = df.drop('schedule', axis=1)

In [ ]:
# Split lat_lng into 2 columns
split_columns = df['lat_lng'].str.split(', ', expand=True)
df['latitude'] = split_columns[0]
df['longitude'] = split_columns[1]
df = df.drop('lat_lng', axis=1)

In [20]:
# Split type details into 2 columns
df['type_details'] = df['type_details'].str.extract(r'(\bFirst flight: \d{4}, .*)$', expand=False)
details = df['type_details'].str.split(', ', expand=True)
df['first_flight'] = details[0].str.extract(r'(\d{4})', expand=False)
df['engine'] = details[1]

In [48]:
df = df.drop('type_details', axis=1)

In [21]:
# Split fatalities from ASN
fatalities = df['fatalities'].str.split(' / ', expand=True)
df['fatalities'] = fatalities[0].str.extract(r'(\d+)')
df['occupants'] = fatalities[1].str.extract(r'(\d+)')

### Merge common columns

In [24]:
df['date'] = df['date_x']
df = df.drop(['date_x', 'date_y'], axis=1)

In [41]:
df['operator'] = df['operator'].fillna(df['owner'])
df = df.drop('owner', axis=1)

In [25]:
df['type'] = df['aircraft_type'].fillna(df['type'])
df = df.drop('aircraft_type', axis=1)

In [26]:
df['yom'] = df['yom'].fillna(df['year_of_manufacture'])
df = df.drop('year_of_manufacture', axis=1)

In [27]:
df['aircraft_flying_hours'] = df['aircraft_flying_hours'].fillna(df['total_airframe_hrs'])
df = df.drop('total_airframe_hrs', axis=1)

In [28]:
df['aircraft_flight_cycles'] = df['aircraft_flight_cycles'].fillna(df['cycles'])
df = df.drop('cycles', axis=1)

In [29]:
df['msn'] = df['msn_x'].fillna(df['msn_y'])
df = df.drop(['msn_x', 'msn_y'], axis=1)

In [30]:
df['flight_phase'] = df['flight_phase'].fillna(df['phase'])
df = df.drop('phase', axis=1)

In [31]:
df['flight_type'] = df['flight_type'].fillna(df['nature'])
df = df.drop('nature', axis=1)

In [32]:
df['departure'] = df['departure_airport'].fillna(df['departure'])
df = df.drop('departure_airport', axis=1)

In [33]:
df['arrival'] = df['destination_airport'].fillna(df['arrival'])
df = df.drop('destination_airport', axis=1)

In [35]:
on_board = df['crew_on_board'] + df['pax_on_board']
on_board.isnull().sum()

29

In [36]:
on_board = df['crew_on_board'] + df['pax_on_board']
df['occupants'] = on_board.fillna(df['occupants'])
df = df.drop(['crew_on_board', 'pax_on_board'], axis=1)

In [37]:
df['fatalities'] = df['total_fatalities'].fillna(df['fatalities'])
df = df.drop(['crew_fatalities', 'pax_fatalities', 'total_fatalities'], axis=1)

In [38]:
df['other_fatalities'] = df['other_fatalities_x'].fillna(df['other_fatalities_y'])
df = df.drop(['other_fatalities_x', 'other_fatalities_y'], axis=1)

#### Summary

In [58]:
null_values = df.isnull().sum()
null_values

operator                            0
registration                      539
flight_phase                      226
flight_type                        23
site                              224
yom                              2183
flight_number                   18480
country                             0
region                              0
captain_flying_hours            14601
captain_flying_hours_on_type    15582
copilot_flying_hours            19121
copilot_flying_hours_on_type    19348
aircraft_flying_hours           15594
aircraft_flight_cycles          19747
type                                0
engine_model                    14627
fatalities                          0
aircraft_damage                  9607
category                         9742
investigating_agency            18314
confidence_rating               15969
location                            0
departure                        5085
arrival                          7116
latitude                           18
longitude   

In [43]:
# Get proportion of rows with null values
nan_rows = df[df.isna().any(axis=1)]

f'{len(nan_rows) / len(df):0%}'

'98.214370%'

In [61]:
# Get proportion of null values for each columns
null_values_ratios = null_values / len(df)
null_values_ratios

operator                        0.000000
registration                    0.025328
flight_phase                    0.010620
flight_type                     0.001081
site                            0.010526
yom                             0.102580
flight_number                   0.868380
country                         0.000000
region                          0.000000
captain_flying_hours            0.686105
captain_flying_hours_on_type    0.732202
copilot_flying_hours            0.898501
copilot_flying_hours_on_type    0.909168
aircraft_flying_hours           0.732766
aircraft_flight_cycles          0.927917
type                            0.000000
engine_model                    0.687327
fatalities                      0.000000
aircraft_damage                 0.451436
category                        0.457779
investigating_agency            0.860580
confidence_rating               0.750388
location                        0.000000
departure                       0.238946
arrival         

### Drop rows and columns

#### Drop redundant/unnecessary columns

survivors: can get it with the number of occupants minus the number of fatalities<br>
date_str: was used for merging dataframes<br>
time: there's already a datetime column<br>
first_flight: it's the first flight of the aircraft in general, not the one involved in the accident<br>
flight_number, registration, msn, investigating_agency: too many unique values, won't help categorize crash<br>
captain_flying_hours, captain_flying_hours_on_type, copilot_flying_hours, copilot_flying_hours_on_type: too many null values<br>

In [74]:
df.columns

Index(['operator', 'registration', 'flight_phase', 'flight_type', 'site',
       'yom', 'flight_number', 'country', 'region', 'captain_flying_hours',
       'captain_flying_hours_on_type', 'copilot_flying_hours',
       'copilot_flying_hours_on_type', 'aircraft_flying_hours',
       'aircraft_flight_cycles', 'type', 'engine_model', 'fatalities',
       'aircraft_damage', 'category', 'investigating_agency',
       'confidence_rating', 'location', 'departure', 'arrival', 'latitude',
       'longitude', 'engine', 'occupants', 'date', 'msn', 'other_fatalities'],
      dtype='object')

### Impute missing values

In [71]:
df['registration'].isnull().sum()

539

In [68]:
# Inpute missing other_fatalities to 0
df['other_fatalities'] = df['other_fatalities'].fillna(0)
assert df['other_fatalities'].isna().sum() == 0

In [73]:
df['flight_phase'].sort_values().unique()

array(['Approach', 'En route', 'Flight', 'Initial climb', 'Landing',
       'Landing (descent or approach)',
       'Manoeuvring  (airshow, firefighting, ag.ops.)', 'Parking',
       'Standing', 'Take off', 'Takeoff (climb)', 'Taxi', 'Taxiing',
       'Unknown', nan], dtype=object)

In [ ]:
# Assert there are no more null values
assert df.isna().sum().sum() == 0

### Convert columns

In [ ]:
# Convert flight_phase, flight_type, site and region to non-ordinal category
columns = ['engine_model', 'category', 'flight_phase', 'flight_type', 'site', 'region']
df[columns] = df[columns].astype('category')

In [ ]:
categories = ['Substantial', 'Destroyed']

df['aircraft_damage'] = pd.Categorical(df['aircraft_damage'], categories, ordered=True)

In [ ]:
# Convert float columns to integer
columns = df.select_dtypes(include=[float]).columns
df[columns] = df[columns].astype('int')

### Export data

In [ ]:
df.columns

In [ ]:
# Reorder columns
df = df[[
	'date',
	'type',
	'registration',
	'msn',
	'yom',
	'engine',
	'engine_model',
	'aircraft_flying_hours',
	'aircraft_flight_cycles',
	'category',
	'flight_phase',
	'flight_type',
	'departure',
	'destination',
	'site',
	'location',
	'country',
	'region',
	'latitude',
	'longitude',
	'aircraft_damage',
	'occupants',
	'fatalities',
	'other_fatalities',
	'investigating_agency'
  ]]

In [ ]:
# Sort data from the earliest to the latest crash
df = df.sort_values(by='date')

In [ ]:
# Reset index
df = df.reset_index(drop=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34956 entries, 0 to 34955
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              34956 non-null  datetime64[ns]
 1   aircraft_type     34956 non-null  object        
 2   operator          34956 non-null  object        
 3   flight_phase      34956 non-null  category      
 4   flight_type       34956 non-null  category      
 5   survivors         34956 non-null  bool          
 6   site              34956 non-null  category      
 7   location          34956 non-null  object        
 8   country           34956 non-null  object        
 9   region            34956 non-null  category      
 10  crew_on_board     34956 non-null  int64         
 11  crew_fatalities   34956 non-null  int64         
 12  pax_on_board      34956 non-null  int64         
 13  pax_fatalities    34956 non-null  int64         
 14  other_fatalities  3495

In [ ]:
# Serialize data with pickle
with open('data/crashes_cleaned_data.pkl', 'wb') as handle:
  pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Export data to CSV
df.to_csv('data/crashes_cleaned_data.csv', index=False)

## End